In [1]:
%load_ext autoreload
%autoreload 2
import os
import glob
import pickle
import numpy as np

In [2]:
from aSPEM import Analysis
import bayesianchangepoint as bcp

In [3]:
list_sujet = ['AM','BMC','CS','DC','FM','IP','LB','OP','RS','SR','TN','YK']
nb_sujet = len(list_sujet)

In [4]:
liste = []
for fname in glob.glob('data/*pkl'):
    a = fname.split('/')[1].split('.')[0].split('_')
    liste.append(a)
            
BET = {}
for x in range(len(liste)) :
    if liste[x][0]=='pari' and liste[x][1] in list_sujet:
        a = 'data/%s_%s_%s_%s.pkl'%(liste[x][0], liste[x][1],liste[x][2],liste[x][3])
        with open(a, 'rb') as fichier :
            b = pickle.load(fichier, encoding='latin1')
            BET[liste[x][1]] = b

e = Analysis(observer='AM', mode='enregistrement')
Full = e.Full_list(modes_bcp=None)

# Scaling Data

In [5]:
file = os.path.join('parametre', 'Data_Scaling.pkl')
with open(file, 'rb') as fichier:
    new_data = pickle.load(fichier, encoding='latin1')

In [6]:
new_bet_full = new_data['new_bet_full']
new_bet_sujet = new_data['new_bet_sujet']
new_va_full = new_data['new_va_full']
new_va_sujet = new_data['new_va_sujet']

# Variation de h

In [7]:
from lmfit import  Model, Parameters
from lmfit import minimize

In [8]:
def fct_BCP(x, tau) :

    h = 1/tau
    p_bar, r_bar, beliefs = bcp.inference(x, h=h, p0=.5, r0=1.)
    p_hat, r_hat = bcp.readout(p_bar, r_bar, beliefs, mode='mean', p0=.5, fixed_window_size=40)#, mode='mean')
    
    return p_hat

In [9]:
def KL_distance(p_data, p_hat):
    distance = p_hat * np.log2(p_hat) - p_hat * np.log2(p_data + 1.*(p_data==0.))
    distance += (1-p_hat) * np.log2(1-p_hat) - (1-p_hat) * np.log2(1-p_data + 1.*(p_data==1.))
    return distance

def residual(params, x, data):
    tau = params['tau']
    model = fct_BCP(x, tau)
    return KL_distance(data, model)

def fit(h, x, bet, va):
    
    tau = 1/h
    x = np.array(x)
    bet = np.array(bet)
    va = np.array(va)


    params = Parameters()
    params.add('tau', value=tau, min=1)

    result_res = minimize(residual, params, args=(x, bet), nan_policy='omit')
    result_v_ant = minimize(residual, params, args=(x, va), nan_policy='omit')
    
    h_bet = 1/result_res.params['tau'].value
    h_va = 1/result_v_ant.params['tau'].value
    
    return h_bet, h_va

In [10]:
def fit_h(new_bet, new_va) :
    
    h_bet, h_va = {}, {}

    for l in ['pause', 'block', 'sujet'] :
        h_bet[l], h_va[l] = {}, {}

    for x, sujet in enumerate(list_sujet) :

        print(sujet)

        prob_sujet, bet_sujet, v_anti_sujet = [], [], []

        p = BET[sujet]['p']
        tau = BET[sujet]['N_trials']/5.
        h = 1./tau 

        for l in ['pause', 'block', 'sujet'] :
            h_bet[l][sujet] = []
            h_va[l][sujet] = []

        for block in range(BET[sujet]['N_blocks']):
            
            bet = new_bet[sujet][block]
            va = new_va[sujet][block]

            prob_block = p[:, block, 0]
            h_b, h_v = fit(h, prob_block, bet, va)

            h_bet['block'][sujet].append(h_b)
            h_va['block'][sujet].append(h_v)

            prob_sujet.extend(p[:, block, 0])
            bet_sujet.extend(bet)
            v_anti_sujet.extend(va)
            #----------------------------------------------------
            # Pour chaque pause !
            #----------------------------------------------------
            liste = [0,50,100,150,200]
            for a in range(len(liste)-1) :
                va_p = va[liste[a]:liste[a+1]]
                bet_p = bet[liste[a]:liste[a+1]]
                prob_pause = p[liste[a]:liste[a+1], block, 0]

                h_b, h_v = fit(h, prob_pause, bet_p, va_p)

                h_bet['pause'][sujet].append(h_b)
                h_va['pause'][sujet].append(h_v)


        h_b, h_v = fit(h, prob_sujet, bet_sujet, v_anti_sujet)
        h_bet['sujet'][sujet].append(h_b)
        h_va['sujet'][sujet].append(h_v)
    return h_bet, h_va

In [11]:
h_bet_full, h_va_full = fit_h(new_bet_full, new_va_full)

AM
BMC
CS
DC
FM
IP
LB
OP
RS
SR
TN
YK


In [12]:
h_bet_sujet, h_va_sujet = fit_h(new_bet_sujet, new_va_sujet)

AM
BMC
CS
DC
FM
IP
LB
OP
RS
SR
TN
YK


In [13]:
list_h = {}
list_h['h_bet_full'] = h_bet_full
list_h['h_va_full'] = h_va_full
list_h['h_bet_sujet'] = h_bet_sujet
list_h['h_va_sujet'] = h_va_sujet

In [14]:
file = os.path.join('parametre', 'list_h_TAU.pkl')
with open(file, 'wb') as fichier:
    f = pickle.Pickler(fichier)
    f.dump(list_h)